# Prepare the data

In [1]:
import datashader as ds
from datashader.bokeh_ext import InteractiveImage
from functools import partial
from datashader.utils import export_image
from datashader.colors import colormap_select, Greys9, Hot, inferno
from datashader import transfer_functions as tf

import numpy as np
import visutil

In [2]:
df = visutil.get_eigenvectors(eigenvectors="C:/Users/haoyu/Documents/Python Scripts/diffusion_map_dev/fake_eigens.npy")
sample = visutil.get_random_samples(sample_num=2000, dim0='eigvec_0', dim1='eigvec_1', eigvec_holder=df)
#sample.head()

In [3]:
from bokeh.models import BoxZoomTool
from bokeh.plotting import figure, output_notebook, show

output_notebook()

x_range, y_range = ((0,1), (0,1))

plot_width  = 600
plot_height = 600

def base_plot(tools='pan,wheel_zoom,reset',plot_width=plot_width, plot_height=plot_height, **plot_args):
    p = figure(tools=tools, plot_width=plot_width, plot_height=plot_height,
        x_range=x_range, y_range=y_range, outline_line_color=None,
        min_border=0, min_border_left=0, min_border_right=0,
        min_border_top=0, min_border_bottom=0, **plot_args)
    
    p.axis.visible = True
    p.xgrid.grid_line_color = None
    p.ygrid.grid_line_color = None
    
    p.add_tools(BoxZoomTool(match_aspect=True))
    
    return p

Loading BokehJS ...

In [7]:
x_range, y_range = ((0,1), (0,1))

background = 'black'
export = partial(export_image, export_path="export", background=background)
cm = partial(colormap_select, reverse=(background=="blue"))
p = base_plot(background_fill_color=background)

g = base_plot()
g.circle(source=sample, x='x', y='y', size=3,
              line_color="navy", fill_color="green", alpha=1)

def create_image(x_range, y_range, w=plot_width, h=plot_height):
    cvs = ds.Canvas(plot_width=w, plot_height=h, x_range=x_range, y_range=y_range)
    agg = cvs.points(source=df,x='eigvec_0', y='eigvec_1')
    img = tf.shade(agg, cmap=inferno)
    img = tf.stack(img, g)
    return tf.dynspread(img, threshold=0.5, max_px=4)

InteractiveImage(p, create_image)

TypeError: Expected `Image`, got: `<class 'bokeh.plotting.figure.Figure'>`

In [8]:
import numpy as np
import holoviews as hv

from bokeh.io import show
from bokeh.layouts import layout
from bokeh.models import Slider, Button

renderer = hv.renderer('bokeh').instance(mode='server')

# Create the holoviews app again
def sine(phase):
    xs = np.linspace(0, np.pi*4)
    return hv.Curve((xs, np.sin(xs+phase))).options(width=800)

stream = hv.streams.Stream.define('Phase', phase=0.)()
dmap = hv.DynamicMap(sine, streams=[stream])

# Define valid function for FunctionHandler
# when deploying as script, simply attach to curdoc
def modify_doc(doc):
    # Create HoloViews plot and attach the document
    hvplot = renderer.get_plot(dmap, doc)

    # Create a slider and play buttons
    def animate_update():
        year = slider.value + 0.2
        if year > end:
            year = start
        slider.value = year

    def slider_update(attrname, old, new):
        # Notify the HoloViews stream of the slider update 
        stream.event(phase=new)
        
    start, end = 0, np.pi*2
    slider = Slider(start=start, end=end, value=start, step=0.2, title="Phase")
    slider.on_change('value', slider_update)
    
    callback_id = None

    def animate():
        global callback_id
        if button.label == '► Play':
            button.label = '❚❚ Pause'
            callback_id = doc.add_periodic_callback(animate_update, 50)
        else:
            button.label = '► Play'
            doc.remove_periodic_callback(callback_id)
    button = Button(label='► Play', width=60)
    button.on_click(animate)
    
    # Combine the holoviews plot and widgets in a layout
    plot = layout([
    [hvplot.state],
    [slider, button]], sizing_mode='fixed')
    
    doc.add_root(plot)
    return doc

# To display in the notebook
show(modify_doc, notebook_url='localhost:8888')

# To display in a script
#    doc = modify_doc(curdoc()) 

INFO:bokeh.server.server:Starting Bokeh server version 0.13.0 (running on Tornado 4.5.3)


INFO:tornado.access:200 GET /autoload.js?bokeh-autoload-element=d367ff4c-f4b0-4308-a216-a0d42eaca750&bokeh-absolute-url=http://localhost:53635&resources=none (::1) 169.00ms
INFO:tornado.access:101 GET /ws?bokeh-protocol-version=1.0&bokeh-session-id=k4WKZxGyHLGZwgPIsqRpdPXHafXV4XnEs8C5hA0PgU8T (::1) 0.00ms
INFO:bokeh.server.views.ws:WebSocket connection opened
INFO:bokeh.server.views.ws:ServerConnection created
